In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Datasets

In [2]:
# obteniendo los datasets
transacciones_final = pd.read_parquet(r'C:\Users\Agricola Guapa\Documents\PROYECTO_MOVii\ProyectoMovii\app\data\transacciones_final.parquet')
usuarios_movii = pd.read_parquet(r'C:\Users\Agricola Guapa\Documents\PROYECTO_MOVii\ProyectoMovii\app\data\usrs_movii.parquet')
# Dataset para las coordenadas de las ciudades,  recurso: https://simplemaps.com
dfcol = pd.read_excel(r'C:\Users\Agricola Guapa\Documents\PROYECTO_MOVii\ProyectoMovii\app\data\mapa_colombia.xlsx')
# Merge
df_merge = transacciones_final.merge(usuarios_movii, how='left', on='usr_id')
df_merge.rename(columns={'labels_x':'etiqueta_transaccion','labels_y':'etiqueta_usuarios', 'id':'id_transaccion'}, inplace = True)
df_merge.drop(columns=['year','month'], inplace=True)
# Se filtrara df_merge

In [30]:
def filtro_resumen(startdate, enddate, product):
    filtro1 = df_merge[df_merge['Anomaly'] == True][['fecha_transaccion', 'nombre_producto','tipo_deposito', 'producto', 'sub_producto','movilizado', 'trx_activo', 'nro_transaccion', 'etiqueta_transaccion', 'Anomaly', 'Anomaly_Score']]
    startdate = "2021-02-01"
    enddate = '2021-02-02'
    product = 'CB'
    startdate = pd.to_datetime(startdate, format="%Y-%m-%d")
    enddate = pd.to_datetime(enddate, format="%Y-%m-%d")
    filtro2 = filtro1[(filtro1['fecha_transaccion'] >= startdate) & (filtro1['fecha_transaccion'] <= enddate)]
    filtro3 = filtro2[filtro2['producto'] == product]
    num_anomalos_filtrados = filtro3.Anomaly.sum()
    filtro3 =filtro3[['tipo_deposito','sub_producto', 'movilizado', 'trx_activo', 'nro_transaccion','etiqueta_transaccion', 'Anomaly_Score']]
    # Nuero de anomalas 
    filtro4 = filtro3.groupby(['sub_producto'])['movilizado','Anomaly_Score','nro_transaccion'].sum()
    filtro4 = filtro4.sort_values(by='movilizado', ascending = False)
    filtro4 = filtro4.reset_index()
    filtro5 = filtro4.query('movilizado > 0')
    columns = [i for i in filtro5.columns]
    rows = [[i for i in row] for row in filtro5.T.itertuples()]
    fig = go.Figure(data=[go.Table(
    # columnorder = [1,2,3,4,5,6,7,8,9,10],
    # columnwidth = [60,60,60,60,60,60,60,60,60,60],
    header= dict(
    values = [i for i in filtro5.columns],
    line_color='darkslategray',
    fill_color='royalblue',
    align=['left','center'],
    font=dict(color='white', size=12),
    height=60
    ),
    cells=dict(
    values=filtro5.T.values,
    line_color='darkslategray',
    fill=dict(color=['paleturquoise', 'white']),
    align=['center', 'center'],
    font_size=10,
    height=50)
    )
    ])
    fig.update_layout(autosize=False,width=1500,height=600,margin=dict(l=0,r=0,b=0,t=0,pad=0))
    return fig
filtro_resumen("2021-02-01", "2021-03-01", 'GB')

C:\Users\Agricola Guapa\AppData\Local\Temp\ipykernel_26648\1289413228.py:13: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

